In [ ]:
import pandas as pd
import requests

def get_geographical_info(latitude, longitude, api_key):
    """
    根据经纬度使用Google Maps Geocoding API获取地理位置信息，包括国家、省、市、县等
    
    参数:
    latitude (float): 维度
    longitude (float): 经度
    api_key (str): Google Maps API的密钥
    
    返回值:
    dict: 返回一个字典，包含国家、省、市、县等信息。
    """
    url = "https://maps.googleapis.com/maps/api/geocode/json"
    params = {
        'latlng': f"{latitude},{longitude}",
        'key': api_key
    }
    try:
        response = requests.get(url, params=params, timeout=10)
        if response.status_code == 200:
            results = response.json().get('results', [])
            if results:
                address_info = {
                    'country': None,
                    'province': None,
                    'city': None,
                    'county': None
                }
                for result in results:
                    for component in result.get('address_components', []):
                        if 'country' in component.get('types', []):
                            address_info['country'] = component.get('long_name')
                        if 'administrative_area_level_1' in component.get('types', []):
                            address_info['province'] = component.get('long_name')
                        if 'locality' in component.get('types', []):
                            address_info['city'] = component.get('long_name')
                        if 'administrative_area_level_2' in component.get('types', []):
                            address_info['county'] = component.get('long_name')
                return address_info
    except requests.RequestException as e:
        return {'error': str(e)}
    return {'error': "Location information not found"}

def process_coordinates(file_path, output_file, api_key):
    """
    处理给定文件中的坐标数据，通过API查询每个坐标的地理信息，并将结果保存到新的文件中。

    参数:
    file_path: str - 包含坐标数据的CSV文件的路径。
    output_file: str - 处理结果将保存到这个文件中，包含原始数据和对应的地理信息。
    api_key: str - 使用API查询地理信息时所需的密钥。

    返回值:
    无
    """
    data = pd.read_csv(file_path, sep='\t')
    with open(output_file, 'w', newline='', encoding='utf-8') as file:
        # 初始化输出文件，添加列标题
        file.write('\t'.join(list(data.columns) + ['Country', 'Province', 'City', 'County']) + '\n')
        
        for index, row in data.iterrows():
            geo_info = get_geographical_info(row['Latitude'], row['Longitude'], api_key)
            print(f"Processing Record {index + 1}: {geo_info}")  # 输出处理进度
            
            # 构建包含地理信息的行数据
            row_data = row.tolist() + [geo_info.get('country', ''),
                                       geo_info.get('province', ''),
                                       geo_info.get('city', ''),
                                       geo_info.get('county', '')]
            # 将每行数据和地理信息写入输出文件，并确保数据立即写入磁盘
            file.write('\t'.join(map(str, row_data)) + '\n')
            file.flush()  # 确保数据实时写入

    print("All records have been processed and saved.")

# 示例使用
api_key = ''  # 替换为您的Google Maps API密钥
file_path = 'C:/Users/LuzHu/Desktop/1.txt'  # 替换为您的CSV文件路径
output_file = 'C:/Users/LuzHu/Desktop/结果.txt'  # 输出文件路径
process_coordinates(file_path, output_file, api_key)


Processing Record 1: {'country': 'Japan', 'province': 'Tokyo', 'city': 'Tokyo', 'county': None}
Processing Record 2: {'country': 'Japan', 'province': 'Niigata', 'city': 'Sado', 'county': None}
Processing Record 3: {'country': 'China', 'province': 'Yunnan', 'city': 'Xishuangbanna Dai Autonomous Prefecture', 'county': None}
Processing Record 4: {'country': 'China', 'province': 'Sichuan', 'city': 'Chengdu', 'county': None}
Processing Record 5: {'country': 'China', 'province': 'Sichuan', 'city': 'Chengdu', 'county': None}
Processing Record 6: {'country': 'China', 'province': 'Guangxi', 'city': 'Nanning', 'county': None}
Processing Record 7: {'country': 'China', 'province': 'Guangxi', 'city': 'Hechi', 'county': None}
Processing Record 8: {'country': 'China', 'province': 'Guangxi', 'city': 'Nanning', 'county': None}
Processing Record 9: {'country': 'China', 'province': 'Liaoning', 'city': 'Shenyang', 'county': None}
Processing Record 10: {'country': 'China', 'province': 'Inner Mongolia', 'ci

In [2]:
import pandas as pd
df = pd.read_csv('1.txt',sep='\t',encoding='utf-8')
df.head()

,ID,Latitude,Longitude
0,HRR1397951,24.975173,107.534202
1,HRR1397952,24.975173,107.534202
2,HRR1397950,24.975173,107.534202
3,HRR1397949,24.975173,107.534202
4,HRR1456898,30.310000,118.510000


In [5]:
df['Temp'] = df['Latitude'].astype(str) + df['Longitude'].astype(str)


In [9]:
df_temp = df.loc[:,['Latitude','Longitude','Temp']]
df_temp.drop_duplicates(subset=('Temp'),inplace=True)
df_temp.to_csv('1_temp.txt',sep='\t',index=False)

In [11]:
df_temp = pd.read_csv('结果_temp.txt',sep='\t')
df_temp

,Latitude,Longitude,Temp,Country,Province,City,County
0,24.975173,107.534202,24.9751734107.534202,China,Guangxi,Hechi,NaN
1,30.310000,118.510000,30.31118.51,China,Anhui,Xuancheng,NaN
2,25.100000,107.660000,25.1107.66,China,Guangxi,Hechi,NaN
3,34.670000,111.810000,34.67111.81,China,Henan,Sanmenxia,NaN


In [13]:
df = df.merge(df_temp,on='Temp',how='left')
df

,ID,Latitude_x,Longitude_x,Temp,Latitude_y,Longitude_y,Country,Province,City,County
0,HRR1397951,24.975173,107.534202,24.9751734107.534202,24.975173,107.534202,China,Guangxi,Hechi,NaN
1,HRR1397952,24.975173,107.534202,24.9751734107.534202,24.975173,107.534202,China,Guangxi,Hechi,NaN
2,HRR1397950,24.975173,107.534202,24.9751734107.534202,24.975173,107.534202,China,Guangxi,Hechi,NaN
3,HRR1397949,24.975173,107.534202,24.9751734107.534202,24.975173,107.534202,China,Guangxi,Hechi,NaN
4,HRR1456898,30.310000,118.510000,30.31118.51,30.310000,118.510000,China,Anhui,Xuancheng,NaN
...,...,...,...,...,...,...,...,...,...,...
489,HRR1956811,36.400000,116.590000,36.4116.59,NaN,NaN,NaN,NaN,NaN,NaN
490,HRR1506149,36.880000,118.350000,36.88118.35,NaN,NaN,NaN,NaN,NaN,NaN
491,HRR1506165,36.880000,118.350000,36.88118.35,NaN,NaN,NaN,NaN,NaN,NaN
492,HRR1506161,36.880000,118.350000,36.88118.35,NaN,NaN,NaN,NaN,NaN,NaN
